In [39]:
pip install pennylane

In [40]:
#same encoding : using angle embedding
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import AngleEmbedding
from sklearn.preprocessing import StandardScaler

In [41]:
n_samples = 100
X = np.column_stack((
    np.random.normal(34, 2, n_samples), # first feature
    np.random.normal(4, 1, n_samples), # Second feature
    np.random.normal(0, 2, n_samples), # Third feature
))

y = np.random.choice([0, 1], size=n_samples)

In [42]:
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
y_scaled = 2 * (y - 0.5)

In [43]:
for k, (x, y) in enumerate(zip(X_scaled, y_scaled)):
  if k <= 10:
      print(f"x = {x}, y = {y}")

x = [-0.21288978 -0.49681849 -0.60797484], y = 1.0
x = [ 0.51200602  0.70015601 -0.33537602], y = 1.0
x = [-1.01941893  0.11774128 -0.62074393], y = 1.0
x = [-0.90158435  0.41622257  0.48411425], y = -1.0
x = [ 0.66007249 -0.3421209  -2.1829589 ], y = 1.0
x = [ 0.82713242 -0.02669606 -0.57721843], y = 1.0
x = [-0.64541526 -0.56488567 -1.07164801], y = -1.0
x = [-0.65682242 -1.51148302  0.14992365], y = -1.0
x = [0.50455085 0.62736618 0.86412728], y = 1.0
x = [-0.68591136  0.10253442 -0.88435563], y = 1.0
x = [-0.30057734 -1.90091841 -0.80264698], y = -1.0


In [44]:
num_qubits = X_scaled.shape[1]

dev = qml.device("default.qubit", wires=num_qubits)

In [45]:
# Define the quantum circuit
@qml.qnode(dev)

def feature_map(x):# Embed the data into the circuit
  qml.AngleEmbedding(x, wires=range(num_qubits), rotation='Y') # Return the full probability vector
  return qml.probs(wires=range(num_qubits))

def quantum_kernel(X1, X2):# kernel_matrix will call our feature_map QNode under the hood
  return qml.kernels.kernel_matrix(X1, X2, kernel=lambda x, y: np.dot(feature_map(x), feature_map(y)))

X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

X_train_pl = np.array(X_scaled)
X_test_pl = np.array(X_test_scaled)

# Compute the kernel matrices
K_train = quantum_kernel(X_train_pl, X_train_pl)
K_test = quantum_kernel(X_test_pl, X_train_pl)

In [46]:
#Train the svm
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

y_train= np.array(y_scaled)
svm = SVC (kernel='precomputed')
svm.fit (K_train, y_train)

#Evaluate the SVM
train_accuracy = svm.score(K_train, y_train)
print(f"Train Accuracy: {train_accuracy:.4f}")

Train Accuracy: 0.5300


In [47]:
test_accuracy = svm.score(K_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.5000
